# 1.introduction

This project is about car classification for [stanford car dataset](https://ai.stanford.edu/~jkrause/cars/car_dataset.html). The Cars dataset contains 16,185 images of 196 classes of cars. The data is split into 8,144 training images and 8,041 testing images, where each class has been split roughly in a 50-50 split. Classes are typically at the level of Make, Model, Year, e.g. 2012 Tesla Model S or 2012 BMW M3 coupe.

It is difficult to directly train deep learning model on this dataset because the limited number of images. Thus we decide to use transfer learning, a common approch used in deep learning to utilize the pretrained model on [imagenet](http://www.image-net.org/) and fine-tune on our own dataset, i.e. car dataset.

This project can show you how to train and fine-tune a deep learning model using kera (tensorflow backend).

![](https://ai.stanford.edu/~jkrause/cars/class_montage.jpg)

## 1.1 Transfer Learning
Transfer learning is one of the most widely used technologies in deep learning and computer vision. If you are not familar with the concept of transfer learning, please refer to our course materials.

![](https://cdn-images-1.medium.com/max/2000/1*9GTEzcO8KxxrfutmtsPs3Q.png)

# 2.Import & read data

# 3.Build model

## 3.1 Load packages

The keras and tensorflow have been pre-installled on colab, and we do not need re-install these packages.

In [4]:
from keras.preprocessing import image
from keras.applications import vgg16
from keras.applications import vgg19
from keras.applications import resnet50
from keras.applications import inception_v3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Model,Sequential
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
# import argparse
from time import time

from skimage import exposure, color

from keras import backend as K
K.set_image_dim_ordering('tf')

C:\Users\Troy\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 3.2 Initalize model
In this step, we will initalize our architecture. We use predefined architectures such as resnet50/vgg19/inception, you can also use your own architectures.

In [1]:
def init_model(train_dir, val_dir, batch_size=32, model_name='resnet50', num_class=196, img_size=224):
    """
    initialize cnn model and training and validation data generator
    parms:
        args: parsed commandline arguments
    return:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
    """
    

    print('loading the model and the pre-trained weights...')

    # load base model
    if model_name == 'vgg19':
        base_model = vgg19.VGG19(include_top=False, weights='imagenet', input_shape = (img_size, img_size, 3)) # need specify input_shape
        # this preprocess_input is the default preprocess func for given network, you can change it or implement your own 
        # use inception_v3 preprocess for vgg16, it seems that it works better than vgg16.preprocess_input
        preprocess_input = vgg19.preprocess_input

    if model_name == 'resnet50':
        base_model = resnet50.ResNet50(weights='imagenet',input_shape=(img_size,img_size,3))
        preprocess_input = resnet50.preprocess_input
    # initalize training image data generator
    # you can also specify data augmentation here
    train_datagen = image.ImageDataGenerator(
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        # samplewise_center=True,
        # samplewise_std_normalization=True,
        # rescale=1./255,
        preprocessing_function=preprocess_input,
        # rotation_range=30,
        # shear_range=0.1,
        # zoom_range=0.1,
        # vertical_flip=True,
        horizontal_flip=True
        )

    # initalize validation image data generator
    # you can also specify data augmentation here
    validation_datagen = image.ImageDataGenerator(
        # samplewise_center=True,
        # samplewise_std_normalization=True
        # rescale=1./255
        preprocessing_function=preprocess_input # preprocess_input
        )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')     

    validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        # color_mode='grayscale',  # 'rgb'
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

    # fix base_model layers
    for layer in base_model.layers:
        layer.trainable = False

    # added some customized layers for your own data
    x = base_model.output
    if model_name == 'vgg19':
        x = GlobalAveragePooling2D(name='avg_pool')(x)
        # x = Flatten(name='flatten')(x)
        # x = Dense(512, activation='relu', name='fc1-pretrain')(x)
        x = Dense(256, activation='relu', name='fc2-pretrain')(x)
        x = Dropout(0.3, name='dropout')(x)
    if model_name == 'resnet50':
        x = GlobalAveragePooling2D(name='avg_pool')(x)
        # x = Flatten(name='flatten')(x)
        # x = Dense(512, activation='relu', name='fc1-pretrain')(x)
        x = Dense(256, activation='relu', name='fc2-pretrain')(x)
        x = Dropout(0.8, name='dropout')(x)

    # added softmax layer
    predictions = Dense(num_class, activation='softmax', name='predictions')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam()
    # model.compile(loss='mean_squared_error', optimizer=sgd)
    model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

    return model, train_generator, validation_generator

## 3.3 train model
This part defines the function to train our model. We can specify the training parameters such as number of epochs, optimizers, and etc here.

In [2]:
def train(model, train_generator, validation_generator, num_class=196, model_name='resnet50', batch_size=32, epochs=30, suffix='laioffer'):
    """
    train the model
    parms:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
        args: parsed command line arguments
    return:
    """
    # define number of steps/iterators per epoch
    stepsPerEpoch = train_generator.samples / batch_size
    validationSteps= validation_generator.samples / batch_size

    # save the snapshot of the model to local drive
    pretrain_model_name = 'pretrained_{}_{}_{}_{}.h5'.format(model_name, num_class, epochs, suffix)
    # visualize the training process
    tensorboard = TensorBoard(log_dir="logs/{}_pretrain_{}".format(model_name, time()), histogram_freq=0, write_graph=True)
    checkpoint = ModelCheckpoint(pretrain_model_name, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    earlystopping = EarlyStopping(monitor='acc', patience=5)
    callbacks_list = [checkpoint, tensorboard, earlystopping]

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps)
    return history

In [3]:
model, train_generator, validation_generator = init_model(train_dir='./data/train_small', model_name='resnet50', val_dir='./data/test_small', num_class=10)

train_generator.samples

loading the model and the pre-trained weights...


NameError: name 'resnet50' is not defined

## 3.4 Fine-tune model
This part defines the function to fine-tune the pre-trained model on our datset. If you do not have enough data, you may consider fine-tune less layers.

In [4]:
 def fine_tune(model, train_generator, validation_generator, num_class=196, model_name='resnet50', batch_size=32, epochs=50, suffix='laioffer'):
    """
    fine tune the model
    parms:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
        args: parsed command line arguments
    return:
    """
    # for specific architectures, define number of trainable layers
    if model_name == 'vgg19':
        trainable_layers = 6

    for layer in model.layers[:-1*trainable_layers]:
        layer.trainable = False

    for layer in model.layers[-1*trainable_layers:]:
        layer.trainable = True

    finetune_model_name = 'finetuned_{}_{}_{}_{}.h5'.format(model_name, num_class, epochs, suffix)
    tensorboard = TensorBoard(log_dir="logs/{}_finetune_{}".format(model_name, time()), histogram_freq=0, write_graph=True)
    checkpoint = ModelCheckpoint(finetune_model_name, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    earlystopping = EarlyStopping(monitor='acc', patience=5)
    callbacks_list = [checkpoint, tensorboard, earlystopping]

    model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),metrics=["accuracy"])

    stepsPerEpoch = train_generator.samples / batch_size
    validationSteps= validation_generator.samples / batch_size
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps)
    return history

# 4.Experiment

## 4.1.Train/fine-tune model
We can start to train and fine-tune our model here.

In [7]:
# initialize model
model, train_generator, validation_generator = init_model(train_dir='./data/train_small', model_name='vgg19', val_dir='./data/test_small', num_class=10)
# pretrain model
train(model, train_generator, validation_generator, num_class=10,model_name='vgg19', epochs=15)
# fine-tune model
history = fine_tune(model, train_generator, validation_generator, num_class=10, model_name='vgg19', epochs=30)

loading the model and the pre-trained weights...
Found 406 images belonging to 10 classes.
Found 403 images belonging to 10 classes.
Epoch 1/15
13/12 [==============================] - 10s 807ms/step - loss: 6.6799 - acc: 0.1850 - val_loss: 3.4145 - val_acc: 0.2928

Epoch 00001: val_acc improved from -inf to 0.29280, saving model to pretrained_vgg19_10_15_laioffer.h5
Epoch 2/15
13/12 [==============================] - 6s 426ms/step - loss: 3.3880 - acc: 0.3594 - val_loss: 2.1889 - val_acc: 0.3772

Epoch 00002: val_acc improved from 0.29280 to 0.37717, saving model to pretrained_vgg19_10_15_laioffer.h5
Epoch 3/15
13/12 [==============================] - 6s 428ms/step - loss: 2.0571 - acc: 0.5165 - val_loss: 1.7242 - val_acc: 0.4739

Epoch 00003: val_acc improved from 0.37717 to 0.47395, saving model to pretrained_vgg19_10_15_laioffer.h5
Epoch 4/15
13/12 [==============================] - 6s 435ms/step - loss: 1.3051 - acc: 0.5857 - val_loss: 1.5906 - val_acc: 0.4988

Epoch 00004: val_ac

KeyboardInterrupt: 

## 4.2 Results

### 4.2.1 Accuracy

In [ ]:
scores = model.evaluate_generator(validation_generator,steps=len(validation_generator))
print("Validation accuracy = ", scores[1])

### 4.2.2 Classification report

In [ ]:
import numpy as np
# predicted_label_probs = model.predict_generator(validation_generator, verbose=1)
# predicted_labels = np.argmax(predicted_label_probs, axis=1)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

test_datagen = image.ImageDataGenerator(
    preprocessing_function=vgg19.preprocess_input # preprocess_input
    )

test_generator = test_datagen.flow_from_directory(
    "./data/test_small",
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,    # keep data in same order as labels
    class_mode='categorical')# only data, no labels

true_labels = test_generator.classes

predicted_label_probs = model.predict_generator(test_generator, verbose=1,steps=len(test_generator))
predicted_labels = np.argmax(predicted_label_probs, axis=1)
# predicted_labels 

In [ ]:
label_map = (test_generator.class_indices)
# label_map

In [ ]:
# print(confusion_matrix(true_labels, predicted_labels))
print(classification_report(true_labels, predicted_labels))
# print("Accuracy = ", accuracy_score(true_labels, predicted_labels))

### 4.2.3 Visualize confusion matrix

In [ ]:
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
cnf_matrix = confusion_matrix(true_labels, predicted_labels)
# Plot non-normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=list(label_map.keys()),
#                       title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix[:10,:10], classes=list(label_map.keys())[:10], normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 4.2.4 Visualize training history

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
cm = confusion_matrix(true_labels, predicted_labels)
print(cm)


plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# predicted_labels

### 4.2.5 Save model

In [ ]:
model.save("model.h5")

### 4.2.6 Load model

In [ ]:
from keras.models import load_model
new_model = load_model("model.h5")

In [ ]:
new_predicted_label_probs = new_model.predict_generator(test_generator, verbose=1,steps=len(test_generator))
new_predicted_labels = np.argmax(new_predicted_label_probs, axis=1)

In [ ]:
print(confusion_matrix(true_labels, predicted_labels))
print(classification_report(true_labels, predicted_labels))
print("Accuracy = ", accuracy_score(true_labels, predicted_labels))